### Spark String functions
Reference: https://github.com/DeltaOptimist/Flight-delays-analysis-using-sparksql-pyspark/tree/main

In [ ]:
!pip install pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [ ]:
#Initialize Spark Session
spark = SparkSession.builder.appName("Flights Analysis").getOrCreate()

#Loading flights dataset
flights = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/content/departuredelays.csv")

In [ ]:
#Create a temp view of the data
flights.createOrReplaceTempView("flightsview")

In [ ]:
#Evaluating the data
val1 = spark.sql("""SELECT * FROM flightsview LIMIT 10""")
val1.show()

+-------+-----+--------+------+-----------+
|   date|delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
|1011245|    6|     602|   ABE|        ATL|
|1020600|   -8|     369|   ABE|        DTW|
|1021245|   -2|     602|   ABE|        ATL|
|1020605|   -4|     602|   ABE|        ATL|
|1031245|   -4|     602|   ABE|        ATL|
|1030605|    0|     602|   ABE|        ATL|
|1041243|   10|     602|   ABE|        ATL|
|1040605|   28|     602|   ABE|        ATL|
|1051245|   88|     602|   ABE|        ATL|
|1050605|    9|     602|   ABE|        ATL|
+-------+-----+--------+------+-----------+



In [ ]:
#1. Average delay of origin airport
avg_delay1 = spark.sql("""
SELECT origin, AVG(delay) as avg_delay
from flightsview
GROUP BY origin
ORDER BY avg_delay desc
LIMIT 10
""")
avg_delay1.show()

+------+------------------+
|origin|         avg_delay|
+------+------------------+
|   GUM| 33.87777777777778|
|   LSE|26.532467532467532|
|   MQT| 23.87012987012987|
|   EGE| 20.57012542759407|
|   ROA|19.885106382978723|
|   MDW|19.657658556043078|
|   BTV|  18.7246192893401|
|   ORD|18.588917606028524|
|   IAD| 18.40343803056027|
|   SCE| 17.91616766467066|
+------+------------------+



In [ ]:
#2. Total flights and avg delay of the week
flights_by_day = spark.sql("""
SELECT date, COUNT(*) as total_flights,
AVG(delay) as avg_delay
FROM flightsview
GROUP BY date
ORDER BY date
LIMIT 7
""")
flights_by_day.show()

+-------+-------------+------------------+
|   date|total_flights|         avg_delay|
+-------+-------------+------------------+
|1010005|            1|              -8.0|
|1010010|            1|              -6.0|
|1010020|            2|              -1.0|
|1010023|            1|              14.0|
|1010025|            2|              15.0|
|1010029|            1|              49.0|
|1010030|            3|-5.666666666666667|
+-------+-------------+------------------+



In [ ]:
#3. Top 5 routes with highest total delays
top_delayed_routes = spark.sql("""
  SELECT origin, destination, SUM(delay) as total_delay,
  COUNT(*) as flight_count
  FROM flightsview
  GROUP BY origin, destination
  ORDER BY total_delay DESC
  LIMIT 5
""")
top_delayed_routes.show()

+------+-----------+-----------+------------+
|origin|destination|total_delay|flight_count|
+------+-----------+-----------+------------+
|   LAX|        SFO|      51844|        3198|
|   ORD|        SFO|      41653|        1731|
|   SFO|        LAX|      40798|        3232|
|   LGA|        ATL|      35761|        2500|
|   JFK|        LAX|      35755|        2720|
+------+-----------+-----------+------------+



In [ ]:
#4. Monthly Flight Trends
monthly_trends = spark.sql("""
SELECT
SUBSTRING(CAST(date as STRING),5,2) as month,
COUNT(*) as total_flights,
AVG(delay) as avg_delay
FROM flightsview
GROUP BY SUBSTRING(CAST(date AS STRING),5,2)
ORDER BY month
""")
monthly_trends.show()

+-----+-------------+------------------+
|month|total_flights|         avg_delay|
+-----+-------------+------------------+
|   00|        30649|11.788965382231067|
|   01|        25780|11.812063615205586|
|   02|        22895|13.687311640096091|
|   03|        25564|13.048623063683305|
|   04|        21493|13.328013771925743|
|   05|        23497|13.285057666936204|
|   10|        26978| 13.16543109200089|
|   11|        23701|12.136070208008102|
|   12|        20706|11.687192118226601|
|   13|        19877|11.790461337223928|
|   14|        19595|12.156723653993366|
|   15|        22111|12.806883451675636|
|   20|        21945| 11.77717019822283|
|   21|        18080|11.789546460176991|
|   22|        15428|  13.0869198859217|
|   23|        17676|11.292939579090293|
|   24|        15752|11.935627221940072|
|   25|        16961|11.581451565355817|
|   30|        18012|11.682378414390406|
|   31|        17022|12.303900834214545|
+-----+-------------+------------------+
only showing top

In [ ]:
#5. Percentage of Delayed Flights by origin
delayed_percentage = spark.sql("""
SELECT
origin,
COUNT(*) as total_flights,
SUM(CASE WHEN delay>0 THEN 1 ELSE 0 END) as delayed_flights,
(SUM(CASE WHEN delay>0 THEN 1 ELSE 0 END)*100.00/COUNT(*)) AS delayed_percentage
FROM flightsview
GROUP BY origin
ORDER BY delayed_percentage DESC
LIMIT 10
""")
delayed_percentage.show()

+------+-------------+---------------+------------------+
|origin|total_flights|delayed_flights|delayed_percentage|
+------+-------------+---------------+------------------+
|   DAL|        11272|           7153| 63.45812633073101|
|   MDW|        20056|          12671| 63.17810131631432|
|   HOU|        14740|           8639| 58.60922659430122|
|   DEN|        53148|          30760| 57.87611951531572|
|   BWI|        21558|          12448| 57.74190555710177|
|   OAK|        10026|           5423| 54.08936764412527|
|   LSE|          154|             83| 53.89610389610390|
|   ORD|        64228|          33812| 52.64370679454444|
|   ISP|         1370|            720| 52.55474452554745|
|   STL|        12142|           6250| 51.47422170976775|
+------+-------------+---------------+------------------+



In [ ]:
#6. Average delay by hour of day
delay_by_hour = spark.sql("""
SELECT
CAST(SUBSTRING(CAST(date AS STRING), 10,2) AS INT) as hour,
AVG(delay) as avg_delay
FROM flightsview
GROUP BY SUBSTRING(CAST(date AS STRING), 10, 2)
ORDER BY hour
""")
delay_by_hour.show()

+----+------------------+
|hour|         avg_delay|
+----+------------------+
|NULL|12.079802928761449|
+----+------------------+

